In [ ]:
import pygame
import sys
import random
import tkinter as tk
from tkinter import simpledialog
from tkinter import messagebox

# Pygameの初期化
pygame.init()

# ウィンドウサイズの設定
screen = pygame.display.set_mode((300, 430))

# ウィンドウタイトルの設定
pygame.display.set_caption("AIペナルティーシュート対決")

# サッカーボールの位置を初期化
ball_pos = [150, 215]
ball_radius = 10
ball_speed = 1


class Shooter:
    def __init__(self, name):
        self.name = name
        self.stamina = 100
        self.root = tk.Tk()
        self.root.withdraw()

    def shoot(self, round_num, current_score):
        direction = simpledialog.askstring("入力", f"第{round_num}ラウンドです。現在のスコアは{current_score}です。\nシュートの方向を選択してください（左上、右上、左下、右下、中央）:",
                                           parent=self.root)
        speed = simpledialog.askinteger("入力", "シュートの速度を選択してください（15-30）:", parent=self.root)
        if speed > self.stamina:
            messagebox.showinfo("情報", "十分なスタミナがありません。シュートの速度が減少します。")
            speed = self.stamina
        self.stamina -= speed
        accuracy = max(0, 30 - speed)
        if random.randint(0, 30) > accuracy:
            direction = random.choice(['左上', '右上', '左下', '右下', '中央'])
            messagebox.showinfo("情報", f"シュートの方向が変わりました。実際のシュートの方向は {direction} です。")
        return direction, speed


class GoalKeeper:
    def __init__(self):
        self.special_ability = True

    def defend(self, direction, speed):
        use_special_ability = self.special_ability and random.random() > 0.5
        if use_special_ability:
            self.special_ability = False
            messagebox.showinfo("情報", "ゴールキーパーが特殊能力を使用しました！")
            return direction, speed
        else:
            defense_direction = random.choice(['左上', '右上', '左下', '右下', '中央'])
            defense_speed = random.randint(15, 30)
            return defense_direction, defense_speed


class PenaltyShootout:
    def __init__(self, shooter, keeper):
        self.shooter = shooter
        self.keeper = keeper
        self.shooter_score = 0
        self.keeper_score = 0

    def round(self, round_num):
        shot_direction, shot_speed = self.shooter.shoot(round_num, f"{self.shooter_score} - {self.keeper_score}")
        animate_penalty(shot_direction, shot_speed)

        defend_direction, defend_speed = self.keeper.defend(shot_direction, shot_speed)

        if shot_direction == defend_direction and shot_speed <= defend_speed:
            self.keeper_score += 1
            messagebox.showinfo("情報", f"ゴールキーパーが成功した！シュートの方向: {shot_direction}, シュートの速度: {shot_speed}.")
        else:
            self.shooter_score += 1
            messagebox.showinfo("情報", f"{self.shooter.name}が得点しました！シュートの方向: {shot_direction}, シュートの速度: {shot_speed}.")

    def play(self):
        for i in range(5):
            self.round(i + 1)

        if self.shooter_score > self.keeper_score:
            messagebox.showinfo("情報", f"おめでとう！ {self.shooter.name} が勝ちました！最終スコアは {self.shooter_score} - {self.keeper_score} です。")
        elif self.shooter_score < self.keeper_score:
            messagebox.showinfo("情報", f"ゴールキーパーが勝ちました！最終スコアは {self.shooter_score} - {self.keeper_score} です。")
        else:
            messagebox.showinfo("情報", "引き分け！")

        pygame.quit()

def draw_soccer_field():
    # サッカーコートを描画
    pygame.draw.rect(screen, (0, 128, 0), (0, 0, 300, 430))

    # サッカーコートの中央線を描画
    pygame.draw.line(screen, (255, 255, 255), (0, 215), (300, 215), 2)

    # 上下のゴールの位置とサイズを定義
    top_goal_position = ((screen.get_width() - 140) // 2, 0, 140, 60)
    bottom_goal_position = ((screen.get_width() - 140) // 2, screen.get_height() - 60, 140, 60)

    # ゴールを描画
    pygame.draw.rect(screen, (255, 255, 255), top_goal_position)  # 上のゴール
    pygame.draw.rect(screen, (255, 255, 255), bottom_goal_position)  # 下のゴール

    # サッカーボールを白色で描画
    pygame.draw.circle(screen, (0, 0, 0), ball_pos, ball_radius)

def animate_penalty(shot_direction, shot_speed):
    # 上のゴールの5つの特定の位置を定義
    goal_pos = ((screen.get_width() - 140) // 2, 0, 140, 60)
    left_upper_corner = [goal_pos[0], goal_pos[1]]
    right_upper_corner = [goal_pos[0] + goal_pos[2], goal_pos[1]]
    middle = [goal_pos[0] + goal_pos[2] // 2, goal_pos[1] + goal_pos[3] // 2]
    left_lower_corner = [goal_pos[0], goal_pos[1] + goal_pos[3]]
    right_lower_corner = [goal_pos[0] + goal_pos[2], goal_pos[1] + goal_pos[3]]

    # シュートの方向を判定
    if shot_direction == '左上':
        target_pos = left_upper_corner
    elif shot_direction == '右上':
        target_pos = right_upper_corner
    elif shot_direction == '中央':
        target_pos = middle
    elif shot_direction == '左下':
        target_pos = left_lower_corner
    elif shot_direction == '右下':
        target_pos = right_lower_corner

    dx = target_pos[0] - ball_pos[0]
    dy = target_pos[1] - ball_pos[1]
    distance = (dx ** 2 + dy ** 2) ** 0.5
    steps = int(distance / shot_speed)
    sx = dx / steps
    sy = dy / steps

    # ゲームのメインループ
    for _ in range(steps):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # ウィンドウを閉じるボタンをクリックしたらゲームを終了
                pygame.quit()
                sys.exit()

        # 画面をクリア
        screen.fill((0, 0, 0))
        draw_soccer_field()

        # ゴールに到達していなければ、ゴール方向に移動
        ball_pos[0] += sx
        ball_pos[1] += sy

        # 画面を更新
        pygame.display.flip()
        pygame.time.wait(100)  # アニメーションが速すぎないように一定の時間待機

    # ボールの位置をリセット
    ball_pos[0] = 150
    ball_pos[1] = 215

shooter = Shooter("プレイヤー")
keeper = GoalKeeper()
game = PenaltyShootout(shooter, keeper)
game.play()
